In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from collections import Counter
import pandas as pd
from bs4 import BeautifulSoup
import lxml.html
import requests

In [ ]:
''' ROLE DES IMPORTATIONS
BeautifulSoup: RECUPERER LES INFORMATIONS DANS UNE PAGE
webdriver: AGIR SUR LA PAGE (click, entre du texte, ...)
'''

In [2]:
lien = "https://www.indeed.fr/"
metier=["developpeur", "data scientist", "data analyst", "business intelligence"]
contrat = ["CDI", "CDD", "freelance"]
lieux = ["Paris", "Lyon", "Toulouse", "Nantes", "Bordeaux"]
scrap = ["titre", "nom_boite", "adresse_boite", "poste_boite","publication_date"]

In [3]:

metier_recherche = ["data scientist"]
contrat_recherche = ["CDI"]
lieu_recherche = ["Lyon"]

# ouverture de la page et du lien
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(lien)
driver.maximize_window()
# insertion des métiers recherchés
for i in metier_recherche:
    time.sleep(0.2)
    driver.find_element_by_id("text-input-what").send_keys(i, ", ")
# insertion des lieux recherchés
for i in lieu_recherche:
    time.sleep(0.2)
    driver.find_element_by_id("text-input-where").send_keys(i, ", ")    
driver.find_element_by_class_name("icl-WhatWhere-button").click()

nb_lign = []
# obtenir les données voulues et se déplacer de page en page
first_page = True
while True:   
    time.sleep(1)
    # fermeture de la fenetre supérieure si elle apparait
    try:
        driver.find_element_by_class_name("popover-close-link").click()
    except:
        None
    # obtention des propositions faites par la page
    propositions = [i for i in driver.find_elements_by_class_name('jobsearch-SerpJobCard')]
    columns = ["titre", "entreprise", "localisation", "type emploi", "description poste", "salaire", "nb jours"]
    df = pd.DataFrame(columns = columns) 
    for i in propositions[:4]:
        i.click()
        time.sleep(0.2)
        description = driver.find_elements_by_class_name("jobMetadataHeader-itemWithIcon-label")
        L = [description[0].text,'','']
        # test
        page = requests.get(driver.current_url)
        bs = BeautifulSoup(page.content, features='lxml')
        print(bs.find_all('span',attrs={"class":"copany"}))
        print('')
        #print(bs.find_all('span',attrs={"class":"jobMetadataHeader-itemWithIcon-label"}))
            #for p in i.find_all('span',attrs={"class":"jobMetadataHeader-itemWithIcon-label"}):
            #print(i.text)
        
    #    for i in description:
     #       print(i.text)
     #   print(len(description))
     #   print(" \n")
        try:
            c = [
                driver.find_element_by_id("vjs-jobtitle").text,   # id: vjs-jobtitle   titre job
                driver.find_element_by_id("vjs-cn").text,   # id: vjs-cn  entreprise
                L[0],  # id: vjs-loc  localisation class: jobMetadataHeader-itemWithIcon-label)
                L[1], # type emploi (CDI) (class: jobMetadataHeader-itemWithIcon-label)  jobSectionHeader
                driver.find_element_by_id("vjs-desc").text, # id: vjs-desc    description poste
                L[2], # salaire
                driver.find_element_by_id("vjs-footer").text, # id: info-footer   il y a .. jours
                ]   
            df_line = pd.DataFrame([c], columns=columns)
            df = pd.concat([df,df_line], ignore_index=True)  
        except:
            print('', end='')
    break
    # passer sur la page suivante (si elle existe) sinon, sortir
    change = [i for i in driver.find_elements_by_class_name('np')]
    if first_page or len(change)==2:
        [i for i in driver.find_elements_by_class_name('np')][-1].click()
    else: 
        break
    first_page = False
df.head()

   


[WDM] - Looking for [chromedriver 81.0.4044.138 win32] driver in cache 
[WDM] - File found in cache by path [C:\Users\Utilisateur\.wdm\drivers\chromedriver\81.0.4044.138\win32\chromedriver.exe]


[]

[]

[]

[]



,titre,entreprise,localisation,type emploi,description poste,salaire,nb jours
0,CDI - Data scientist senior H/F,BIOMERIEUX sa,Craponne (69),,Acteur mondial dans le domaine du diagnostic i...,,il y a 19 jours\n- Continuer pour postuler\nSi...
1,Data Analyst-Scientist F/H,BIRDZ,Lyon 7e (69),,Intitulé du Poste : Data Analyst-Data Scientis...,,Apec.fr - il y a 19 jours\n- Continuer pour po...
2,Data Analyst-Data Scientist H/F,BIRDZ,Lyon 7e (69),,"1. NOTRE IDENTITE\nChez Birdz, nous avons la c...",,Il y a plus de 30 jours\nSignaler un abus
3,Data scientist – Machine learning,Davidson consulting,Lyon (69),,Pour le compte d’un de nos clients grands comp...,,DAVIDSON - Il y a plus de 30 jours\n- Continue...
